In [1]:
# read in the necessary packages
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split


In [3]:
# read in the dataset
dataset = pd.read_excel('dataset2.xlsx',converters={'Medical result（0 No；1 mild；2 severe）':int})

# fill in the missing value with 0
col_set = dataset.columns
for column in col_set :
    dataset[column].fillna(0, inplace = True)

# split the target variable and trainning features
X = dataset.iloc[:,:-1]
y = dataset['Medical result（0 No；1 mild；2 severe）']

In [4]:
# split training set and test set
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y_train = np_utils.to_categorical(Y_train)
dummy_y_test = np_utils.to_categorical(Y_test)

In [5]:
# define neural network model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(11, input_dim=47, activation='relu'))
	model.add(Dense(3, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [6]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)
clf = estimator.fit(X_train,dummy_y_train)

y_predicted = clf.model.predict(X_test)

In [7]:
y_output = []
for each in y_predicted:
    a,b,c = each
    if max(a,b,c) == a:
        output = 0
    elif max(a,b,c) == b:
        output = 1
    else:
        output = 2
    y_output.append(output)

In [8]:
# prediction accuracy
np.mean(y_output == Y_test)

0.7789473684210526

In [9]:

from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
precision_recall_fscore_support(Y_test, y_output, average=None)

(array([0.82926829, 0.89534884, 0.58730159]),
 array([0.80952381, 0.70642202, 0.94871795]),
 array([0.81927711, 0.78974359, 0.7254902 ]),
 array([ 42, 109,  39], dtype=int64))